<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/fi_whisper_medium_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-agb7ly6m
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-agb7ly6m
  Resolved https://github.com/huggingface/transformers to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8119970 sha256=05ec20edc46bc81ed072db2ce53a9b5322ae984e330139dc8a9a65715354c0cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-5o0dc75w/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [5]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "fi", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 478
})

In [7]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=41c06da97ac349af90d2f538a1e1b0e60ebce94f1620afe39a5c667230c10f02
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [32]:
model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 131MiB/s]


In [33]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [34]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [35]:
text

,0
0,kiitän budjettivaliokuntaa ja sen varjoesittel...
1,syynä on myös puutteellinen tiedotus.
2,arvoisa puhemies indonesian valtiollinen ideol...
3,euroopan investointipankki sai pääosin sen mit...
4,ulkoasiainhallinto voi ja sen tulee pyrkiä ets...
...,...
473,meidän on myös kehitettävä vaihtoehtoisia mate...
474,suurin osa seminaariin osallistujista oli kuit...
475,jos ympäristövaliokunnan tahto menisi läpi eur...
476,haluan kiittää varjoesittelijöitä rakentavasta...


In [36]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [37]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,kiitän budjettivaliokuntaa ja sen varjoesittel...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000732421875, -0.00079345703125, -0.000213...",16000
1,male,syynä on myös puutteellinen tiedotus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002532958984375, -0.00750732421875, -0.006...",16000
2,male,arvoisa puhemies indonesian valtiollinen ideol...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000579833984375, -0.00054931640625, -0.000...",16000
3,male,euroopan investointipankki sai pääosin sen mit...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.008209228515625, 0.0086669921875, 0.0078125...",16000
4,male,ulkoasiainhallinto voi ja sen tulee pyrkiä ets...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0003662109375, 0.00018310546875, 0.0003356...",16000
...,...,...,...,...,...
473,female,meidän on myös kehitettävä vaihtoehtoisia mate...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, 0.000640869140625, 9.15527...",16000
474,male,suurin osa seminaariin osallistujista oli kuit...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.03009033203125, 0.029571533203125, -0.02673...",16000
475,male,jos ympäristövaliokunnan tahto menisi läpi eur...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002288818359375, 0.001007080078125, -0.0006...",16000
476,male,haluan kiittää varjoesittelijöitä rakentavasta...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, 6.103515625e...",16000


In [38]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,kiitän budjettivaliokuntaa ja sen varjoesittel...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000732421875, -0.00079345703125, -0.000213...",16000
1,male,syynä on myös puutteellinen tiedotus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002532958984375, -0.00750732421875, -0.006...",16000
2,male,arvoisa puhemies indonesian valtiollinen ideol...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000579833984375, -0.00054931640625, -0.000...",16000
3,male,euroopan investointipankki sai pääosin sen mit...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.008209228515625, 0.0086669921875, 0.0078125...",16000
4,male,ulkoasiainhallinto voi ja sen tulee pyrkiä ets...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0003662109375, 0.00018310546875, 0.0003356...",16000
...,...,...,...,...,...
471,male,toivon ettemme päästä valloilleen talouskriisi...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.076904296875, 0.030517578125, 0.03247070312...",16000
474,male,suurin osa seminaariin osallistujista oli kuit...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.03009033203125, 0.029571533203125, -0.02673...",16000
475,male,jos ympäristövaliokunnan tahto menisi läpi eur...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002288818359375, 0.001007080078125, -0.0006...",16000
476,male,haluan kiittää varjoesittelijöitä rakentavasta...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, 6.103515625e...",16000


In [39]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
8,female,nyt olisi ehkä puheenjohtajakauden tilausta si...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0059814453125, -0.00238037109375, -0.00048...",16000
9,female,näitä investointeja on tehtävä kaikissa jäsenv...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0030517578125, -0.0003662109375, 0.0010375...",16000
11,female,arvoisa puhemies hyvät kollegat tuen talous ja...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000152587890625, 0.000152587890625, -0.000...",16000
22,female,vihreät olisivat halunneet viedä työntekijöide...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.08245849609375, -0.0870361328125, -0.08416...",16000
24,female,tähän on ehdottomasti puututtava koska euroopp...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0069580078125, -0.0037841796875, 6.1035156...",16000
...,...,...,...,...,...
460,female,tiedämme että lähtökohta oli huono koska komis...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.000762939453125, -9.1552734375e-05, -0...",16000
462,female,arvoisa puhemies venäjä on eu n tärkeä kauppak...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0009765625, -0.001678466796875, -0.0008850...",16000
469,female,nyt kahdeksan vuotta myöhemmin kreikalla on ve...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00439453125, -0.005279541015625, 0.0051879...",16000
472,female,on välttämätöntä että myös unionin rahankäyttö...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003936767578125, -0.001953125, 0.000457763...",16000


In [40]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="fi")
    return result["text"]

In [41]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [42]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.4
Processed 2 files, WER: 0.2
Processed 3 files, WER: 0.3684210526315789
Processed 4 files, WER: 0.25
Processed 5 files, WER: 0.15384615384615385
Processed 6 files, WER: 0.17647058823529413
Processed 7 files, WER: 0.25925925925925924
Processed 8 files, WER: 0.38461538461538464
Processed 9 files, WER: 0.23076923076923078
Processed 10 files, WER: 0.1111111111111111
Processed 11 files, WER: 0.3
Processed 12 files, WER: 0.2962962962962963
Processed 13 files, WER: 0.3
Processed 14 files, WER: 0.3333333333333333
Processed 15 files, WER: 0.4
Processed 16 files, WER: 0.125
Processed 17 files, WER: 0.2631578947368421
Processed 18 files, WER: 0.16666666666666666
Processed 19 files, WER: 0.5454545454545454
Processed 20 files, WER: 0.3
Processed 21 files, WER: 0.2
Processed 22 files, WER: 0.13636363636363635
Processed 23 files, WER: 0.5
Processed 24 files, WER: 0.34782608695652173
Processed 25 files, WER: 0.16666666666666666
Processed 26 files, WER: 0.19047619047619047
Pr

In [43]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.13333333333333333
Processed 2 files, WER: 0.2631578947368421
Processed 3 files, WER: 0.75
Processed 4 files, WER: 0.4
Processed 5 files, WER: 1.3076923076923077
Processed 6 files, WER: 0.47058823529411764
Processed 7 files, WER: 0.4375
Processed 8 files, WER: 0.23076923076923078
Processed 9 files, WER: 0.3888888888888889
Processed 10 files, WER: 0.2857142857142857
Processed 11 files, WER: 0.2857142857142857
Processed 12 files, WER: 0.2571428571428571
Processed 13 files, WER: 1.0
Processed 14 files, WER: 0.15789473684210525
Processed 15 files, WER: 0.16666666666666666
Processed 16 files, WER: 0.09523809523809523
Processed 17 files, WER: 0.24324324324324326
Processed 18 files, WER: 0.16666666666666666
Processed 19 files, WER: 0.08333333333333333
Processed 20 files, WER: 0.3333333333333333
Processed 21 files, WER: 0.6086956521739131
Processed 22 files, WER: 0.36363636363636365
Processed 23 files, WER: 0.3333333333333333
Processed 24 files, WER: 0.3636363636363636